In [1]:
import src.data_mgmt.RecSysData as rsd
from torch.utils.data import DataLoader
import torch
import pandas as pd
import numpy as np
from src.data_mgmt.BaseDataClass import BaseDataClass as bdc
from src.data_mgmt.ValidationBaseDataClass import ValidationDataClass as vdc
from sklearn.metrics import mean_squared_error as mse

In [2]:
#def _loadUpDf(pathname, filename, extensions):
           
df = bdc._loadUpDf('/Users/andrealevy/cmu_msba_2022_ml_applications_2/data/','train','.json.gz')

Loading JSON into Dataframe: 200000it [00:14, 13784.80it/s]


In [7]:
df.columns

Index(['reviewHash', 'reviewTime', 'reviewText', 'nHelpful', 'outOf',
       'reviewerID', 'reviewHash', 'categories', 'unixReviewTime', 'itemID',
       'rating', 'summary', 'categoryID', 'helpfulPerc',
       'summaryCharacterLength', 'reviewTextCharacterLength', 'parentCategory',
       'cat1', 'cat2', 'cat3', 'cat1_child', 'cat2_parent', 'cat2_child'],
      dtype='object')

In [9]:
sample = len(df)

rawdata = np.zeros((sample, 3), dtype=object)
rawratings = np.zeros(sample)

for i in range(sample):
    rawdata[i,0] = df.iloc[i,5]
    rawdata[i,1] = df.iloc[i,9]
    rawdata[i,2] = df.iloc[i,10]
    rawratings[i] = df.iloc[i,10]

In [23]:
alpha = np.mean(rawratings)

In [11]:
df_new = pd.DataFrame(rawdata, columns = ["reviewerID", "itemID", "rating"])
df_new

,reviewerID,itemID,rating
0,U490934656,I402344648,4.0
1,U714157797,I697650540,4.0
2,U507366950,I464613034,5.0
3,U307862152,I559560885,2.0
4,U742726598,I476005312,5.0
...,...,...,...
199995,U781794983,I245323432,5.0
199996,U151975942,I990230316,1.0
199997,U525354881,I037381245,4.0
199998,U995566285,I343675670,5.0


In [12]:
item_set = []
for i in df_new.iloc[:,1]:
    item_set.append(i)

In [15]:
avg_rating_item = {}

for item in item_set:
    avg_rating_item[item] = np.mean(df_new['rating'][(df_new["itemID"] == item)])

In [16]:
avg_rating_item

{'I402344648': 4.157894736842105,
 'I697650540': 3.588235294117647,
 'I464613034': 4.2,
 'I559560885': 3.75,
 'I476005312': 4.285714285714286,
 'I342631873': 3.975609756097561,
 'I773829721': 5.0,
 'I740935639': 4.571428571428571,
 'I570436467': 4.666666666666667,
 'I025792869': 4.6,
 'I485369965': 4.5,
 'I628647840': 4.0,
 'I237487188': 4.333333333333333,
 'I589840074': 4.666666666666667,
 'I067437283': 4.857142857142857,
 'I504436890': 4.8,
 'I098326083': 3.75,
 'I569542576': 4.291666666666667,
 'I529938024': 3.8,
 'I120637125': 4.0,
 'I600737158': 4.818181818181818,
 'I022811473': 4.285714285714286,
 'I993512356': 4.5,
 'I377043463': 4.6,
 'I551165914': 4.75,
 'I257039108': 3.9473684210526314,
 'I110916183': 4.125,
 'I827990828': 4.4,
 'I968239393': 4.857142857142857,
 'I004989797': 4.0,
 'I780600249': 4.111111111111111,
 'I776577495': 4.0,
 'I101537153': 4.434782608695652,
 'I612515617': 4.818181818181818,
 'I060403419': 4.333333333333333,
 'I069178741': 4.178082191780822,
 'I86427

In [21]:
actualtestset = pd.read_csv('/Users/andrealevy/cmu_msba_2022_ml_applications_2/data/pairs_Purchase.txt')
actualtestset[['reviewerID','itemID']] = actualtestset['reviewerID-itemID'].str.split("-",expand=True)
actualtestset['rating'] = 0

In [25]:
actualtestset_len = len(actualtestset)
predicted_ratings_test = np.zeros(actualtestset_len)

for i in range(actualtestset_len):
    item = actualtestset.iloc[i,3]
    if item in item_set:
        predicted_ratings_test[i] = avg_rating_item[item]
    else:
        predicted_ratings_test[i] = alpha

In [34]:
predicted_ratings_test

array([4.375     , 4.16666667, 3.5       , ..., 4.35714286, 4.25      ,
       4.2       ])

In [27]:
actualtestset['rating'] = predicted_ratings_test
actualtestset

,reviewerID-itemID,prediction,reviewerID,itemID,rating
0,U938994110-I529819131,NaN,U938994110,I529819131,4.375000
1,U181459539-I863471064,NaN,U181459539,I863471064,4.166667
2,U941668816-I684585522,NaN,U941668816,I684585522,3.500000
3,U768449391-I782253949,NaN,U768449391,I782253949,4.400000
4,U640450168-I232683472,NaN,U640450168,I232683472,4.047619
...,...,...,...,...,...
27995,U337041888-I763827121,NaN,U337041888,I763827121,4.636364
27996,U457455307-I242828364,NaN,U457455307,I242828364,3.571429
27997,U052546714-I111529174,NaN,U052546714,I111529174,4.357143
27998,U566804667-I857242737,NaN,U566804667,I857242737,4.250000


In [41]:
for i in range(actualtestset_len):
    if actualtestset.iloc[i,4] >= 4:
        actualtestset.iloc[i,1] = 1
    else:
        actualtestset.iloc[i,1] = 0

In [42]:
actualtestset

,reviewerID-itemID,prediction,reviewerID,itemID,rating
0,U938994110-I529819131,1.0,U938994110,I529819131,4.375000
1,U181459539-I863471064,1.0,U181459539,I863471064,4.166667
2,U941668816-I684585522,0.0,U941668816,I684585522,3.500000
3,U768449391-I782253949,1.0,U768449391,I782253949,4.400000
4,U640450168-I232683472,1.0,U640450168,I232683472,4.047619
...,...,...,...,...,...
27995,U337041888-I763827121,1.0,U337041888,I763827121,4.636364
27996,U457455307-I242828364,0.0,U457455307,I242828364,3.571429
27997,U052546714-I111529174,1.0,U052546714,I111529174,4.357143
27998,U566804667-I857242737,1.0,U566804667,I857242737,4.250000


In [43]:
actualtestset[['reviewerID-itemID','prediction']].to_csv('Andrea-SimplePurchase-2021-12-11v3.csv')